# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_data = pd.read_csv("WeatherPyData.csv")
vacation_data.head()

,lat,long,temp,cloudiness,cloudiness_perct,humidity,wind_speed,city,country
0,28.37,129.48,26.00,broken clouds,75,83,7.70,naze,JP
1,35.60,103.21,4.84,clear sky,0,62,1.15,linxia,CN
2,18.80,110.40,24.47,overcast clouds,96,86,5.88,wanning,CN
3,68.05,166.44,-5.54,overcast clouds,100,94,3.03,bilibino,RU
4,68.53,146.42,-17.31,overcast clouds,100,92,4.10,belaya gora,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmap
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = vacation_data[["lat", "long"]].astype(float)
humidity = vacation_data["humidity"].astype(int)

In [5]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
vacation_location = vacation_data.loc[(vacation_data['temp'] > 20) & 
                                      (vacation_data['cloudiness'].isin(['light rain', 'thunderstorm'])),["lat", "long", "city", "country"]]

In [23]:
vacation_location.dropna()

,lat,long,city,country
46,-0.60,73.08,hithadhoo,MV
54,-21.21,-159.78,avarua,CK
73,-0.91,122.88,biak,ID
77,2.04,45.34,mogadishu,SO
115,-6.22,-57.75,jacareacanga,BR
137,0.52,25.20,kisangani,CD
169,-2.57,150.80,kavieng,PG
242,-16.48,-151.75,faanui,PF
285,5.95,80.54,matara,LK
305,-3.67,152.43,namatanai,PG


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df =vacation_location[["city", "country", "lat", "long"]].copy()
# hotel_df["Hotel Name"] = ""
hotel_df

,city,country,lat,long
46,hithadhoo,MV,-0.60,73.08
54,avarua,CK,-21.21,-159.78
73,biak,ID,-0.91,122.88
77,mogadishu,SO,2.04,45.34
115,jacareacanga,BR,-6.22,-57.75
137,kisangani,CD,0.52,25.20
169,kavieng,PG,-2.57,150.80
242,faanui,PF,-16.48,-151.75
285,matara,LK,5.95,80.54
305,namatanai,PG,-3.67,152.43


In [46]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params = {
    "location": f"{row['lat']}, {row['long']}",
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
    "keyword": "hotel"
}   
    
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    
    try:
        row['hotel name']=results[0]['name']
        
        hotel_df.loc[index, 'hotel name'] = results[0]['name']
        hotel_df.loc[index, 'city'] = results[0]['vicinity']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
hotel_df

------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------


,city,country,lat,long,hotel name
46,"Abuharee Grand, Addu City",MV,-0.60,73.08,Abuharee Grand
54,"Ara Tapu, Avarua District",CK,-21.21,-159.78,Paradise Inn
73,"Bungin Timur, Banggai Regency",ID,-0.91,122.88,RedDoorz near RSUD Kota Luwuk
77,Mogadishu,SO,2.04,45.34,Villa Rose Guest House
115,"Av. Brg. Haroldo Coimbra Veloso, Jacareacanga",BR,-6.22,-57.75,Zé Prata
137,"Boulevard Du 30, Kisangani",CD,0.52,25.20,Lab Hotel
169,"Tanga St, Kavieng",PG,-2.57,150.80,Kavieng Niu Lodge
242,Bora-Bora,PF,-16.48,-151.75,Bora Vaite Lodge
285,"14/4 Ferry Road, Matara",LK,5.95,80.54,OYO 416 Onaro Beach View
305,Namatanai,PG,-3.67,152.43,Namatanai Hotel


In [39]:
results[0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': -16.4737653, 'lng': -151.7404551},
  'viewport': {'northeast': {'lat': -16.47240237010728,
    'lng': -151.7390592701073},
   'southwest': {'lat': -16.47510202989272, 'lng': -151.7417589298927}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Bora Vaite Lodge',
 'photos': [{'height': 692,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103956249885868522751">Vaite Wernert Mana</a>'],
   'photo_reference': 'CmRaAAAArQZ3jgHWwDgok-Mwyo8h5czW19_u3z6Ic-_CCU9qZuy2Vu4Rkl4HcPGaGu9POEixQbu42CqrplxlLUbTYiix931Boz8nIMWzC2EyHYC2ECTEw8aH0SrOQZGtcnSvufFlEhDh2XxsgUxT-4lGsbJwhhpgGhRS6wCwE3DNYLRHigp7rfAJA2hTUw',
   'width': 1200}],
 'place_id': 'ChIJg9sGdxG9vXYRV4gT69Uuofo',
 'plus_code': {'compound_code': 'G7G5+FR Bora-Bora, French Polynesia',
  'global_code': '53MCG7G5+FR'},
 'rating': 5,
 'reference': 'ChIJg9sGdxG9vXYRV4gT69Uuofo',
 'scope': 'GOOGLE',
 'type

In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [52]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))